## 1 xref交叉引用

是androguard分析的主要内容。能够提供谁调用了这个方法，这个方法调用了谁。xref_from xref_to 


dx提供了三种对象的分析工作。包括类、方法和字符串。其中类和方法有被引用和引用关系，字符串只有被引用关系。

analysis.Analysis类下边有三个子类
analysis.ClassAnalysis, analysis.MethodAnalysis, analysis.StringAnalysis三个分析类。dx可以得到各个分析类的列表。针对每一个分析类，都有一系列的方法可以使用。
另外还有一些补充的分析FiledAnalysis等。

### 获取方法的外部参照
```py
In [10]: for meth in dx.classes['Ltests/androguard/TestActivity;'].get_methods():
    ...:     print("inside method {}".format(meth.name))
    ...:     for _, call, _ in meth.get_xref_to():
    ...:         print("  calling -> {} -- {}".format(call.class_name, call.name))
    ...:
inside method testCall1
  calling -> Ljava/lang/StringBuilder; -- toString
  calling -> Ljava/lang/StringBuilder; -- append
  calling -> Ljava/lang/StringBuilder; -- <init>
  calling -> Ljava/io/PrintStream; -- println
inside method testCalls
  calling -> Ljava/lang/Object; -- getClass
  calling -> Ljava/io/PrintStream; -- println
  calling -> Ltests/androguard/TestIfs; -- testIF
  calling -> Ltests/androguard/TestActivity; -- testCall2
[...]
```

### 获取字符串的外部参照

```py
In [14]: for _, meth in dx.strings['boom'].get_xref_from():
    ...:     print("Used in: {} -- {}".format(meth.class_name, meth.name))
    ...:
Used in: Ltests/androguard/TestActivity; -- test_base
```

### 获取字段的外部参照

```py
In [25]: for field in dx.find_fields(classname='Ltests/androguard/TestActivity;', fieldname='^value$'):
    ...:     print("Field: {}".format(field.name))
    ...:     for _, meth in field.get_xref_read():
    ...:         print("  read in {} -- {}".format(meth.class_name, meth.name))
    ...:     for _, meth in field.get_xref_write():
    ...:         print("  write in {} -- {}".format(meth.class_name, meth.name))
    ...:
Field: value
  read in Ltests/androguard/TestActivity; -- pouet
  read in Ltests/androguard/TestActivity; -- test1
  read in Ltests/androguard/TestActivity; -- test_base
  read in Ltests/androguard/TestActivity; -- testVars
  write in Ltests/androguard/TestActivity; -- <init>
  write in Ltests/androguard/TestActivity; -- pouet2
  write in Ltests/androguard/TestActivity; -- <init>
  write in Ltests/androguard/TestActivity; -- <init>
```

## 2 生成控制流图
反编译android源代码，并生成控制流图。
默认会生成所有内部类的控制流图。还会输出反编译的Android代码。

```sh
$ androguard decompile -d output_folder -f jpg --limit "LTestDefaultPackage.*" examples/android/TestsAndroguard/bin/TestActivity.apk
[INFO    ] androguard.analysis: End of creating cross references (XREF)
[INFO    ] androguard.analysis: run time: 0min 00s
Dump information examples/android/TestsAndroguard/bin/TestActivity.apk in output_folder
Create directory output_folder
Decompilation ... End
Dump LTestDefaultPackage$TestInnerClass$TestInnerInnerClass; <init> (LTestDefaultPackage$TestInnerClass; I I)V ... jpg ... source codes ... bytecodes ...
Dump LTestDefaultPackage$TestInnerClass$TestInnerInnerClass; <init> (LTestDefaultPackage$TestInnerClass; I I LTestDefaultPackage$TestInnerClass$TestInnerInnerClass;)V ... jpg ... bytecodes ...
Dump LTestDefaultPackage$TestInnerClass$TestInnerInnerClass; Test (I)V ... jpg ... bytecodes ...
Dump LTestDefaultPackage$TestInnerClass; <init> (LTestDefaultPackage; I I)V ... jpg ... source codes ... bytecodes ...
Dump LTestDefaultPackage$TestInnerClass; <init> (LTestDefaultPackage; I I LTestDefaultPackage$TestInnerClass;)V ... jpg ... bytecodes ...
Dump LTestDefaultPackage$TestInnerClass; access$1 (LTestDefaultPackage$TestInnerClass;)I ... jpg ... bytecodes ...
Dump LTestDefaultPackage$TestInnerClass; Test (I)V ... jpg ... bytecodes ...
Dump LTestDefaultPackage; <init> ()V ... jpg ... source codes ... bytecodes ...
Dump LTestDefaultPackage; main ([Ljava/lang/String;)V ... jpg ... bytecodes ...
```


### 控制流图的实例

![](image/control_flow.jpg)


## 3 指令和字节码

### 获取原始字节码

这里是获取一个方法对应的原始字节码

```py
for method in dx.get_methods():
    if method.is_external():
        continue
    # Need to get the EncodedMethod from the MethodClassAnalysis object
    m = method.get_method()
    if m.get_code():
        # get_code() returns None or a DalvikCode object
        # get_bc() returns a DCode object
        # get_raw() returns bytearray
        print(m.get_code().get_bc().get_raw())
```


### 获取反编译后的源代码


```
m.get_source()
for method in dx.get_methods():
    if method.is_external():
        continue
    m = method.get_method()
    print(m.source())
```




## 4 调用图call_graph

使用命令行生成调用图。

使用gephi读取图像

但问题是，这个图像生成工具貌似存在bug和问题。无法很好的显示结果。

```sh
Usage: androguard cg [OPTIONS] APK

  Create a call graph and export it into a graph format.

  The default is to create a file called callgraph.gml in the current
  directory!

  classnames are found in the type "Lfoo/bar/bla;".

  Example:

      $ androguard cg examples/tests/hello-world.apk

Options:
  -o, --output TEXT           Filename of the output file, the extension is
                              used to decide which format to use  [default:
                              callgraph.gml]
  -s, --show                  instead of saving the graph, print it with
                              mathplotlib (you might not see anything!)
  -v, --verbose               Print more output
  --classname TEXT            Regex to filter by classname  [default: .*]
  --methodname TEXT           Regex to filter by methodname  [default: .*]
  --descriptor TEXT           Regex to filter by descriptor  [default: .*]
  --accessflag TEXT           Regex to filter by accessflags  [default: .*]
  --no-isolated / --isolated  Do not store methods which has no xrefs
  --help                      Show this message and exit.
```